In [ ]:
!pip install PyPDF2
!pip install transformers
from PyPDF2 import PdfReader

from google.colab import drive
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 KB 8.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 15.2 MB/s eta 0:00:00
Mounted at /content/drive


In [ ]:
import os

In [ ]:
import string
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def text1(p):
  doc = PdfReader(p)
  x = len(doc.pages)
  text = ""
  text = doc.pages[x-2].extract_text() + doc.pages[x-1].extract_text()
  text = text.translate (str.maketrans ('', '', string.punctuation))
  y = word_tokenize(text)
  print(y)
  n = len(y)
  y = y[int(n-312):(n+1)]
  y = ' '.join(y)
  return y

In [ ]:
directory_path = '/content/drive/MyDrive/aiforlaw/NER_TRAIN/test data set'
directory_files = os.listdir(directory_path)
df=[]
b=[]
for i in directory_files:
  b=[i,text1(directory_path+"/"+i)]
  df.append(b)

['Concurrent', 'List', 'and', 'a', 'direct', 'conflict', 'is', 'seen', 'If', 'there', 'is', 'a', 'repugnancy', 'due', 'to', 'overlapping', 'found', 'between', 'List', 'II', 'on', 'the', 'one', 'hand', 'and', 'List', 'I', 'and', 'List', 'III', 'on', 'the', 'other', 'the', 'State', 'law', 'will', 'be', 'ultra', 'vires', 'and', 'shall', 'have', 'to', 'give', 'way', 'to', 'the', 'Union', 'law', '3', 'Taxation', 'is', 'considered', 'to', 'be', 'a', 'distinct', 'matter', 'for', 'purposes', 'of', 'legislative', 'competence', 'There', 'is', 'a', 'distinction', 'made', 'between', 'general', 'subjects', 'of', 'legislation', 'and', 'taxation', 'The', 'general', 'subjects', 'of', 'legislation', 'are', 'dealt', 'with', 'in', 'one', 'group', 'of', 'entries', 'and', 'power', 'of', 'taxation', 'in', 'a', 'separate', 'group', 'The', 'power', 'to', 'tax', 'can', 'not', 'be', 'deduced', 'from', 'a', 'general', 'legislative', 'entry', 'as', 'an', 'ancillary', 'power', '4', 'The', 'entries', 'in', 'the', '

In [ ]:

from transformers import AutoModelForSequenceClassification,AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("xlnet-base-cased")
#model = AutoModelForSequenceClassification.from_pretrained("xlnet-base-cased",num_labels=3,ff_activation="gelu")
model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/aiforlaw/NER_TRAIN/xlnet-gelu',local_files_only=True)


Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'logits_proj.weight', 'sequence_summary.summary.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [ ]:
a = get('We therefore allow this appeal, set aside the impugned order of the High Court and quash the criminal proceedings of Criminal Complaint No. 1029 of 2015. We clarify that this will not come in the way of instituting appropriate proceedings in future in case the Civil Court comes to the conclusion that the disputed sale deed dated 29.12.2010 is forged. We refrain ourselves from expressing any opinion as regards the genuineness or otherwise of the sale deed in question as this question is wide open before the Civil Court. The Civil Court shall decide the civil suit pending between the parties on its own merits and on the basis of the evidence that may be led by both the sides. It shall be open to the Civil Court to take the opinion of the hand writing expert as regards the signature of the complainant on the disputed sale deed. We clarify that we have passed the aforesaid order in the facts and circumstances of the present case and the same shall not be cited as a precedent.')

In [ ]:
import torch
import numpy as np
if torch.cuda.is_available():  
 device = torch.device("cuda") 
 print('There are %d GPU(s) available.' % torch.cuda.device_count()) 
 print('We will use the GPU:', torch.cuda.get_device_name(0)) # If not...
else:
 print('No GPU available, using the CPU instead.')
 device = torch.device("cpu")
model.to(device)
def get(text):
    encoding = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=512)
    encoding = {k:v.to(model.device) for k,v in encoding.items()}

    outputs = model( **encoding)

    logits = outputs.logits

    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(logits.squeeze().cpu())
    probs = probs.detach().numpy()
    label = np.argmax(probs, axis=-1)
    
    if label == 1:
        return {
            'w': 'Petitioner',
            'probability': probs[1]
        }
    elif label == 2:
        return {
            'w': 'respondent',
            'probability': probs[2]
            }
    elif label == 0:
        return {
            'w': 'am',
            'probability': probs[0]
            }

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
for i in df:
  print(i[0])
  print(get(i[1]))

C7.PDF
{'w': 'respondent', 'probability': 0.99986947}
C10.PDF
{'w': 'Petitioner', 'probability': 0.99989665}
C9.PDF
{'w': 'Petitioner', 'probability': 0.99988234}
C8.PDF
{'w': 'respondent', 'probability': 0.99984396}
C5.PDF
{'w': 'Petitioner', 'probability': 0.99987674}
C4.PDF
{'w': 'Petitioner', 'probability': 0.99651766}
C1.PDF
{'w': 'respondent', 'probability': 0.99986625}
C3.PDF
{'w': 'Petitioner', 'probability': 0.99979395}
C2.PDF
{'w': 'am', 'probability': 0.9985378}
C6.PDF
{'w': 'Petitioner', 'probability': 0.999764}
